In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Online Synthetic (OS) 方法实现
- 根据当前问题动态生成多个输入-输出样例
- 作为Prompt输入来指导SQL生成
"""

import json
import os
from pathlib import Path
from dataclasses import dataclass
from typing import Dict, Any, List, Tuple
import torch
from tqdm import tqdm
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
)
import re

# ---------- 可调参数 ----------
@dataclass
class Config:
    model_name: str = r"/home/yangliu26/qwen3-8b"  # 请根据实际模型路径调整
    input_json: str = r"/home/yangliu26/data/train/schema_linking_result.json"
    output_dir: str =  r"/home/yangliu26/CHASE/candidates/os_results"
    
    # 文本生成超参
    max_new_tokens: int = 1024
    do_sample: bool = True
    temperature: float = 0.7
    # 性能设置
    batch_size: int = 4
    use_fp16: bool = True
    device_map: str = "auto"
    # OS特定参数
    num_general_examples: int = 3
    num_schema_aware_examples: int = 3

# 配置实例
CFG = Config()

def load_model_and_tokenizer(cfg: Config):
    """加载模型和分词器"""
    # 量化配置
    quant_cfg = None
    if not cfg.use_fp16:
        quant_cfg = BitsAndBytesConfig(load_in_8bit=True)

    tokenizer = AutoTokenizer.from_pretrained(
        cfg.model_name,
        trust_remote_code=True,
        padding_side="left",
    )
    model = AutoModelForCausalLM.from_pretrained(
        cfg.model_name,
        trust_remote_code=True,
        torch_dtype=torch.float16 if cfg.use_fp16 else torch.float32,
        quantization_config=quant_cfg,
        device_map=cfg.device_map,
    )
    return tokenizer, model

def batched(iterable: List[Any], n: int):
    """将列表分批切片"""
    for i in range(0, len(iterable), n):
        yield iterable[i : i + n]

def load_prompt_template(path: str) -> str:
    with open(path, encoding="utf-8") as f:
        return f.read()

def generate_examples_by_sql_features(db_schema: str, num_examples: int, generator) -> List[Tuple[str, str]]:
    # GENERAL_EXAMPLES_TEMPLATE是生成通用示例的提示模板
    GENERAL_EXAMPLES_TEMPLATE = load_prompt_template("/home/yangliu26/CHASE/template/general_examples_template.txt")
    """生成涵盖不同SQL特性的示例"""
    prompt = GENERAL_EXAMPLES_TEMPLATE.format(
        db_schema=db_schema,
        num_examples=num_examples
    )
    
    response = generator(prompt, max_new_tokens=2048, do_sample=True, temperature=0.8)
    text = response[0]["generated_text"].strip()
    
    # 解析示例
    examples = []
    lines = text.split('\n')
    i = 0
    while i < len(lines):
        if lines[i].startswith("问题:"):
            question = lines[i][3:].strip()
            i += 1
            # 寻找SQL行
            while i < len(lines) and not lines[i].startswith("SQL:"):
                i += 1
            if i < len(lines):
                sql = lines[i][4:].strip()
                examples.append((question, sql))
        i += 1
    
    return examples[:num_examples]  # 确保不超过请求的数量

def extract_sql(text: str) -> str:
    """从生成的文本中提取SQL"""
    # 尝试找到SQL:后面的内容
    if "SQL:" in text:
        return text.split("SQL:", 1)[1].strip()
    return text.strip()

def online_synthetic_icl(question: str, db_schema: str, generator, 
                         num_general: int = 3, num_schema_aware: int = 3):
    """主函数：使用在线合成示例方法生成SQL"""
    # 步骤1: 使用常见SQL特征生成通用示例
    general_examples = generate_examples_by_sql_features(db_schema, num_general, generator)
    
    # 步骤2: 使用过滤列生成schema-aware示例
    relevant_columns = filter_columns_by_question(question, db_schema, generator)
    schema_examples = generate_examples_by_schema(db_schema, relevant_columns, num_schema_aware, generator)
    
    # 步骤3: 组合所有示例 + 当前问题进入Prompt
    prompt = format_few_shot_prompt(general_examples + schema_examples, question, db_schema)
    
    # 步骤4: 生成SQL
    response = generator(prompt, max_new_tokens=1024, do_sample=False)
    return extract_sql(response[0]["generated_text"])

In [3]:
# 加载数据
with open(CFG.input_json, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [4]:
item=data[0]

In [5]:
question=item.get("question")
db_schema = item.get("schema_linking")
question, db_schema

('Name movie titles released in year 1945. Sort the listing by the descending order of movie popularity.',
 {'movies': ['movie_title',
   'movies',
   'movie_title_language',
   'movie_release_year',
   'movie_popularity',
   'movie_release_year',
   'movie_title',
   'movies',
   'movie_id',
   'movie_release_year',
   'movies',
   'movie_title',
   'movie_popularity',
   'movies',
   'movie_title',
   'movie_popularity',
   'movies'],
  'ratings': ['movie_id',
   'critic',
   'ratings',
   'critic',
   'ratings',
   'critic_likes',
   'rating_url'],
  'lists': ['lists']})

In [6]:
 # 加载模型和分词器
tokenizer, model = load_model_and_tokenizer(CFG)
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Device set to use cuda:0


In [7]:
# GENERAL_EXAMPLES_TEMPLATE是生成通用示例的提示模板
GENERAL_EXAMPLES_TEMPLATE = load_prompt_template("/home/yangliu26/CHASE/template/general_examples_template.txt")
"""生成涵盖不同SQL特性的示例"""
prompt = GENERAL_EXAMPLES_TEMPLATE.format(
    db_schema=db_schema,
    num_examples=3
)

In [14]:
response = generator(prompt, max_new_tokens=1024, do_sample=True, temperature=0.8)
text = response[0]["generated_text"].strip()

In [15]:
print(text)

Let me first clarify the database structure and schema based on the information provided. The data appears to be a bit inconsistent, with multiple entries for the same column names like'movie_title' and'movies' appearing multiple times. However, based on the context, I'll try to infer the actual schema.

After analyzing the database information, I believe the actual schema is as follows:

**Movies Table**
- movie_id (Primary Key)
- movie_title
- movie_title_language
- movie_release_year
- movie_popularity

**Ratings Table**
- movie_id (Foreign Key to movies)
- critic
- ratings
- critic_likes
- rating_url

**Lists Table**
- list_id (Primary Key)
- list_name (probably, though not explicitly mentioned)
- movie_id (Foreign Key to movies)

However, the schema description is not entirely clear. The repeated column names make it challenging to determine the exact structure. Given this ambiguity, I'll proceed with the most logical interpretation of the schema based on common database practices

In [20]:
# 解析示例
examples = []
lines = text.split('\n')
i = 0
while i < len(lines):
    if lines[i].startswith("Question:"):
        question = lines[i][9:].strip()
        i += 1
        # 寻找SQL行
        while i < len(lines) and not lines[i].startswith("SQL:"):
            i += 1
        if i < len(lines):
            sql = lines[i][4:].strip()
            examples.append((question, sql))
    i += 1
examples

[('List all movies along with their popularity scores.',
  'SELECT movie_title, movie_popularity FROM movies;'),
 ('Find all movies released in the year 2020.',
  'SELECT movie_title FROM movies WHERE movie_release_year = 2020;'),
 ('Show the average popularity of movies rated by critics.',
  'SELECT AVG(movie_popularity) AS average_popularity FROM movies JOIN ratings ON movies.movie_id = ratings.movie_id;'),
 ('What are the top 5 most popular movies?',
  'SELECT movie_title, movie_popularity FROM movies ORDER BY movie_popularity DESC LIMIT 5;'),
 ('Find all movies that have been rated by critics with more than 100 likes.',
  'SELECT movies.movie_title, ratings.critic, ratings.ratings FROM movies JOIN ratings ON movies.movie_id = ratings.movie_id WHERE ratings.critic_likes > 100;'),
 ('Get the total number of movies released each year.',
  'SELECT movie_release_year, COUNT(*) AS total_movies FROM movies GROUP BY movie_release_year;'),
 ('Find the movie with the highest popularity score

In [24]:
SCHEMA_AWARE_TEMPLATE = load_prompt_template("/home/yangliu26/CHASE/template/schema_aware_template.txt")
"""生成基于特定schema的示例"""
prompt = SCHEMA_AWARE_TEMPLATE.format(
    db_schema=db_schema,
    num_examples=3
)
print(prompt)

You are a professional database expert. Your task is to create natural-language questions and matching SQL query examples for the specified database.

**Database information:**
{'movies': ['movie_title', 'movies', 'movie_title_language', 'movie_release_year', 'movie_popularity', 'movie_release_year', 'movie_title', 'movies', 'movie_id', 'movie_release_year', 'movies', 'movie_title', 'movie_popularity', 'movies', 'movie_title', 'movie_popularity', 'movies'], 'ratings': ['movie_id', 'critic', 'ratings', 'critic', 'ratings', 'critic_likes', 'rating_url'], 'lists': ['lists']}

Please produce **3** distinct pairs of natural-language questions and SQL queries. Each pair should focus on the columns listed above and demonstrate a variety of SQL operations.

For each pair, follow this format:

```
Question: [natural-language question]
SQL: [corresponding SQL query]
```

Make sure every SQL statement is accurate and can be executed on the provided database.



In [26]:
response = generator(prompt, max_new_tokens=1024, do_sample=True, temperature=0.8)
text = response[0]["generated_text"].strip()
print(text)

Okay, let me try to figure out how to approach this. The user provided a database schema with tables'movies' and 'ratings', and some columns. But looking at the columns listed for'movies', there's a lot of repetition. For example,'movie_title' appears multiple times, same with'movie_release_year' and'movie_popularity'. That's a bit confusing. Maybe there's a typo or duplication in the column names? But I have to work with what's given.

First, I need to create three distinct pairs of natural-language questions and SQL queries. Each pair should focus on the columns listed. The SQL queries should be accurate and executable. Let me start by understanding the tables.

The'movies' table seems to have columns like movie_title, movie_release_year, movie_popularity, movie_id, and others. Wait, looking at the list, the'movies' columns are repeated. Maybe it's a mistake, but perhaps the actual columns are movie_id, movie_title, movie_release_year, movie_popularity, etc. Maybe the repeated entrie

In [27]:
# 解析示例
examples = []
lines = text.split('\n')
i = 0
while i < len(lines):
    if lines[i].startswith("Question:"):
        question = lines[i][9:].strip()
        i += 1
        # 寻找SQL行
        while i < len(lines) and not lines[i].startswith("SQL:"):
            i += 1
        if i < len(lines):
            sql = lines[i][4:].strip()
            examples.append((question, sql))
    i += 1
examples

[('What are the top 5 most popular movies based on their popularity score?',
  'SELECT movie_title, movie_popularity FROM movies ORDER BY movie_popularity DESC LIMIT 5;'),
 ('What is the average rating for each movie?',
  'SELECT m.movie_title, AVG(r.ratings) AS average_rating FROM movies m JOIN ratings r ON m.movie_id = r.movie_id GROUP BY m.movie_id;'),
 ('Which critic has the highest total number of likes?',
  'SELECT critic, SUM(critic_likes) AS total_likes FROM ratings GROUP BY critic ORDER BY total_likes DESC LIMIT 1;')]

In [34]:
FEW_SHOT_TEMPLATE = load_prompt_template("/home/yangliu26/CHASE/template/os_few_shot_template.txt")
"""格式化few-shot提示"""
examples_text = ""
for i, (q, sql) in enumerate(examples, 1):
    examples_text += f"Example {i}:\nQuestion: {q}\nSQL: {sql}\n"

few_shot = FEW_SHOT_TEMPLATE.format(
    db_schema=db_schema,
    examples=examples_text,
    question=question
)
print(few_shot)

You are a professional database expert. Your task is to convert a natural-language question into an SQL query.

**Database information:**
{'movies': ['movie_title', 'movies', 'movie_title_language', 'movie_release_year', 'movie_popularity', 'movie_release_year', 'movie_title', 'movies', 'movie_id', 'movie_release_year', 'movies', 'movie_title', 'movie_popularity', 'movies', 'movie_title', 'movie_popularity', 'movies'], 'ratings': ['movie_id', 'critic', 'ratings', 'critic', 'ratings', 'critic_likes', 'rating_url'], 'lists': ['lists']}

**Examples:**
Example 1:
Question: What are the top 5 most popular movies based on their popularity score?
SQL: SELECT movie_title, movie_popularity FROM movies ORDER BY movie_popularity DESC LIMIT 5;
Example 2:
Question: What is the average rating for each movie?
SQL: SELECT m.movie_title, AVG(r.ratings) AS average_rating FROM movies m JOIN ratings r ON m.movie_id = r.movie_id GROUP BY m.movie_id;
Example 3:
Question: Which critic has the highest total n

In [40]:
response = generator(few_shot, max_new_tokens=1024, do_sample=False)

In [42]:
print(extract_sql(response[0]["generated_text"]))

SELECT critic, SUM(critic_likes) AS total_likes FROM ratings GROUP BY critic ORDER BY total_likes DESC LIMIT 1;


In [ ]:
def process_data():
    """处理数据并生成SQL"""
    # 创建输出目录
    os.makedirs(CFG.output_dir, exist_ok=True)
    
    # 加载数据
    with open(CFG.input_json, 'r', encoding='utf-8') as f:
        data = json.load(f)
    data = data[0]
    print(data)
    # 加载模型和分词器
    tokenizer, model = load_model_and_tokenizer(CFG)
    generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        return_full_text=False,
    )
    
    results = []
    
    # 处理每个样本
    for item in tqdm(data, desc="处理样本"):
        question = item.get("question", "")
        db_schema = item.get("schema_linking", "")
        
        # 使用OS方法生成SQL
        try:
            generated_sql = online_synthetic_icl(
                question, 
                db_schema, 
                generator,
                num_general=CFG.num_general_examples,
                num_schema_aware=CFG.num_schema_aware_examples
            )
            
            # 保存结果
            result = {
                "id": item.get("id", ""),
                "question": question,
                "db_schema": db_schema,
                "generated_sql": generated_sql,
                "gold_sql": item.get("query", "")  # 原始正确SQL
            }
            results.append(result)
            
        except Exception as e:
            print(f"处理样本时出错: {e}")
    
    # 保存所有结果
    output_path = os.path.join(CFG.output_dir, "os_results.json")
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    
    print(f"处理完成，结果已保存到 {output_path}")

if __name__ == "__main__":
    process_data()